# Porfolio Optimization using Yahoo Finance & PyPortfolioOpt


In [62]:
# package uploads
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [63]:
pip install yahoo-fin

Note: you may need to restart the kernel to use updated packages.


In [64]:
import yahoo_fin.stock_info as si

In [65]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [66]:
import yfinance as yf

In [67]:
pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.


In [68]:
import pandas_datareader as pdr
import datetime as dt

In [69]:
pip install PyPortfolioOpt

Note: you may need to restart the kernel to use updated packages.


## Ten companies in different sectors to depict diversity

In [70]:
# combine stocks function
from functools import reduce

def combine_stocks(tickers):
    data_frames = []
    for i in tickers:
        data_frames.append(get_stock(i))
        
    df_merged = reduce(lambda left, right: pd.merge(left, right, on = ['Date'], how = 'outer'), data_frames)
    print(df_merged.head())
    return df_merged

In [71]:
# get stock function 
def get_stock(ticker):
    data = pdr.DataReader(f"{ticker}", "yahoo", start, end)
    data[f'{ticker}'] = data["Adj Close"]
    data = data[[f'{ticker}']]
    print(data.head())
    return data

In [72]:
# set start & end periods
start = dt.datetime(2000,1,1)
end = dt.datetime.now()

In [73]:
# create portfolio
stocks = ["MA", "UL", "MNST", "MSFT", "ETSY", "UNP", "BRK", "COST", "LMT", "SPGI"]
portfolio = combine_stocks(stocks)

                  MA
Date                
2006-05-25  4.273607
2006-05-26  4.174203
2006-05-30  4.087801
2006-05-31  4.175130
2006-06-01  4.413893
                  UL
Date                
2000-01-03  7.892840
2000-01-04  7.777974
2000-01-05  8.106158
2000-01-06  8.270247
2000-01-07  8.729711
                MNST
Date                
2000-01-03  0.089844
2000-01-04  0.085938
2000-01-05  0.085938
2000-01-06  0.088542
2000-01-07  0.086589
                 MSFT
Date                 
2000-01-03  36.718285
2000-01-04  35.477951
2000-01-05  35.852016
2000-01-06  34.651043
2000-01-07  35.103889
                 ETSY
Date                 
2015-04-16  30.000000
2015-04-17  27.580000
2015-04-20  24.900000
2015-04-21  25.750000
2015-04-22  25.120001
                 UNP
Date                
2000-01-03  7.065440
2000-01-04  7.055190
2000-01-05  7.055190
2000-01-06  7.485885
2000-01-07  7.434612
            BRK
Date           
2000-01-03  6.1
2000-01-04  6.8
2000-01-05  6.9
2000-01-06  6.9
2000-01-

In [74]:
# export portfolio to .csv
portfolio.to_csv(r'C:\Users\Ramzi\OneDrive\MAC_BACKUP\Spring 2022\Financial Analytics\Jupyter Financial\Fard\HWportfolio.csv')

In [75]:
# read back HWportfolio.csv
HWportfolio = pd.read_csv('HWportfolio.csv')

In [76]:
# view dataframe
HWportfolio

,Date,MA,UL,MNST,MSFT,ETSY,UNP,BRK,COST,LMT,SPGI
0,2006-05-25,4.273607,12.975266,7.580000,17.197424,NaN,16.834862,9.27684,38.892891,45.344852,38.151474
1,2006-05-26,4.174203,13.247764,7.632500,17.182940,NaN,16.944576,9.58029,38.914345,45.126469,38.224491
2,2006-05-30,4.087801,12.975266,7.471250,16.770029,NaN,16.611759,9.97044,38.292355,45.496113,37.428604
3,2006-05-31,4.175130,13.056436,7.702917,16.407818,NaN,16.970182,9.79704,37.841934,45.414680,37.676857
4,2006-06-01,4.413893,13.021650,7.656667,16.530970,NaN,17.054306,9.75369,38.771351,45.878273,37.764469
...,...,...,...,...,...,...,...,...,...,...,...
5575,2006-05-18,NaN,13.006184,7.366667,16.538219,NaN,16.729136,10.40390,38.228001,45.581932,38.000938
5576,2006-05-19,NaN,12.887016,7.645833,16.342623,NaN,16.714560,10.23050,38.749912,46.374290,38.000938
5577,2006-05-22,NaN,12.944992,7.430833,16.574444,NaN,16.816633,9.53694,38.685566,46.180870,37.644428
5578,2006-05-23,NaN,13.077051,7.391667,16.509243,NaN,16.758307,9.71034,38.349564,45.382286,37.149673


In [77]:
# define mu & S
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()

In [78]:
# import and setup effcient frontier
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

In [79]:
# weighted positions of stocks in portfolio
cleaned_weights = ef.clean_weights()
print(dict(cleaned_weights))

{'MA': 0.65561, 'UL': 0.0, 'MNST': 0.0, 'MSFT': 0.0, 'ETSY': 0.34439, 'UNP': 0.0, 'BRK': 0.0, 'COST': 0.0, 'LMT': 0.0, 'SPGI': 0.0}


In [80]:
# performance of the porfolio given its efficient weights
ef.portfolio_performance(verbose=True)

Expected annual return: 18.3%
Annual volatility: 165.4%
Sharpe Ratio: 0.10


(0.18331577741899424, 1.6535427591077787, 0.09876719336070659)

In [81]:
# given weights, capital of $10,000, how many shares of each stock we should purchase at the current ask, and how much capital remaining on hand. 
from pypfopt.discrete_allocation import DiscreteAllocation , get_latest_prices

latest_prices = get_latest_prices(portfolio)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = 10000)

allocation, leftover = da.greedy_portfolio()

print("Discrete Allocation: ", allocation)
print("Funds remaining : ${:.2f}".format(leftover))

Discrete Allocation:  {'MA': 20, 'ETSY': 24}
Funds remaining : $283.16


## Ten companies in the same industry (all tech) to disprove diversity

In [82]:
# create non-diversifed portfolio
stocks = ["FB", "GOOG", "PYPL", "MSFT", "INTU", "ANET", "NVDA", "DOCU", "ADBE", "AAPL"]
portfolio = combine_stocks(stocks)

                   FB
Date                 
2012-05-18  38.230000
2012-05-21  34.029999
2012-05-22  31.000000
2012-05-23  32.000000
2012-05-24  33.029999
                 GOOG
Date                 
2004-08-19  49.982655
2004-08-20  53.952770
2004-08-23  54.495735
2004-08-24  52.239197
2004-08-25  52.802086
                 PYPL
Date                 
2015-07-06  36.709999
2015-07-07  36.619999
2015-07-08  34.700001
2015-07-09  34.500000
2015-07-10  34.689999
                 MSFT
Date                 
2000-01-03  36.718285
2000-01-04  35.477936
2000-01-05  35.852016
2000-01-06  34.651039
2000-01-07  35.103874
                 INTU
Date                 
2000-01-03  27.254925
2000-01-04  27.537647
2000-01-05  35.482300
2000-01-06  31.665470
2000-01-07  33.277012
              ANET
Date              
2014-06-06  13.750
2014-06-09  13.800
2014-06-10  15.000
2014-06-11  15.875
2014-06-12  16.500
                NVDA
Date                
2000-01-03  0.895731
2000-01-04  0.871814
2000-01-05  0

In [83]:
# export portfolio to .csv
portfolio.to_csv(r'C:\Users\Ramzi\OneDrive\MAC_BACKUP\Spring 2022\Financial Analytics\Jupyter Financial\Fard\HWportfolio1.csv')

In [84]:
# read back HWportfolio.csv
HWportfolio1 = pd.read_csv('HWportfolio1.csv')

In [92]:
# view dataframe
HWportfolio1

,Date,FB,GOOG,PYPL,MSFT,INTU,ANET,NVDA,DOCU,ADBE,AAPL
0,2012-05-18,38.230000,299.078979,NaN,23.954281,49.742241,NaN,2.773731,NaN,31.309999,16.219666
1,2012-05-21,34.029999,305.908386,NaN,24.347101,50.417526,NaN,2.821950,NaN,32.009998,17.164627
2,2012-05-22,31.000000,299.278229,NaN,24.355293,51.092804,NaN,2.787507,NaN,32.009998,17.032822
3,2012-05-23,32.000000,303.592072,NaN,23.823343,51.530819,NaN,2.856391,NaN,32.180000,17.448421
4,2012-05-24,33.029999,300.702881,NaN,23.790600,51.457806,NaN,2.780619,NaN,31.540001,17.288168
...,...,...,...,...,...,...,...,...,...,...,...
5575,2004-08-12,NaN,NaN,NaN,17.084667,17.307444,NaN,0.771501,NaN,21.430643,0.464375
5576,2004-08-13,NaN,NaN,NaN,17.173656,17.302919,NaN,0.803647,NaN,21.720449,0.471562
5577,2004-08-16,NaN,NaN,NaN,17.218140,17.483864,NaN,0.795994,NaN,22.419979,0.470645
5578,2004-08-17,NaN,NaN,NaN,17.192726,17.664818,NaN,0.839620,NaN,22.370014,0.472021


In [93]:
# redefine mu & S
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage

mu = mean_historical_return(portfolio)
S = CovarianceShrinkage(portfolio).ledoit_wolf()

In [94]:
# import and setup effcient frontier
from pypfopt.efficient_frontier import EfficientFrontier

ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()

In [98]:
# weighted positions of stocks in portfolio
cleaned_weights1 = ef.clean_weights()
print(dict(cleaned_weights1))

{'FB': 0.26288, 'GOOG': 0.0, 'PYPL': 0.03381, 'MSFT': 0.0, 'INTU': 0.0, 'ANET': 0.568, 'NVDA': 0.0, 'DOCU': 0.13532, 'ADBE': 0.0, 'AAPL': 0.0}


In [100]:
# performance of the porfolio given its efficient weights
ef.portfolio_performance(verbose=True)

Expected annual return: 9.3%
Annual volatility: 20.1%
Sharpe Ratio: 0.36


(0.09271477165793861, 0.20143747254400182, 0.3609793686328886)

In [102]:
# given weights, capital of $10,000, how many shares of each stock we should purchase at the current ask, and how much capital remaining on hand. 
from pypfopt.discrete_allocation import DiscreteAllocation , get_latest_prices

latest_prices = get_latest_prices(portfolio)

da = DiscreteAllocation(weights, latest_prices, total_portfolio_value = 10000)

allocation, leftover = da.greedy_portfolio()

print("Discrete Allocation: ", allocation)
print("Funds remaining : ${:.2f}".format(leftover))

Discrete Allocation:  {'ANET': 48, 'FB': 14, 'DOCU': 14, 'PYPL': 4}
Funds remaining : $32.87


## end. 